In [5]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP

# Generate RSA key pair
key = RSA.generate(2048)
private_key = key.export_key()
public_key = key.publickey().export_key()

# Function to encrypt passwords using RSA
def encrypt_password(password, unique_key):
    cipher_rsa = PKCS1_OAEP.new(RSA.import_key(public_key))
    encrypted_password = cipher_rsa.encrypt((password + unique_key).encode())
    return encrypted_password

# Function to decrypt passwords using RSA
def decrypt_password(encrypted_password, unique_key):
    cipher_rsa = PKCS1_OAEP.new(RSA.import_key(private_key))
    decrypted_password = cipher_rsa.decrypt(encrypted_password).decode()
    return decrypted_password[:-len(unique_key)]

# Function to save the encrypted password to a file
def save_password(username, password, unique_key):
    with open('passwords_rsa.txt', 'a') as file:
        encrypted_password = encrypt_password(password, unique_key)
        file.write(f"{username}:{encrypted_password.hex()}:{unique_key}\n")

# Function to retrieve the encrypted password from the file and decrypt it
def get_password(username, unique_key):
    with open('passwords_rsa.txt', 'r') as file:
        for line in file:
            parts = line.strip().split(':')
            if len(parts) == 3:
                if parts[0] == username and parts[2] == unique_key:
                    encrypted_password_hex = parts[1]
                    encrypted_password = bytes.fromhex(encrypted_password_hex)
                    decrypted_password = decrypt_password(encrypted_password, unique_key)
                    return decrypted_password
        return None

# Function to handle saving passwords
def save_password_handler():
    username = username_entry.get()
    password = password_entry.get()
    unique_key = key_entry.get()
    if username and password and unique_key:
        save_password(username, password, unique_key)
        messagebox.showinfo("Success", "Password saved successfully.")
        username_entry.delete(0, tk.END)
        password_entry.delete(0, tk.END)
        key_entry.delete(0, tk.END)
    else:
        messagebox.showerror("Error", "Please enter username, password, and unique key.")

# Function to handle retrieving passwords
def get_password_handler():
    username = username_entry.get()
    unique_key = key_entry.get()
    if username and unique_key:
        password = get_password(username, unique_key)
        if password:
            messagebox.showinfo("Password", f"The password for {username} is: {password}")
        else:
            messagebox.showerror("Error", f"No password found for {username} or invalid unique key.")
    else:
        messagebox.showerror("Error", "Please enter username and unique key.")
        
# Function to show all saved passwords in the table
def show_all_saved_passwords():
    tree.delete(*tree.get_children())  # Clear existing rows
    with open('passwords_rsa.txt', 'r') as file:
        for line in file:
            parts = line.strip().split(':')
            if len(parts) == 3:
                username = parts[0]
                encrypted_password = parts[1]
                tree.insert('','end', values=(username, encrypted_password))

# GUI setup
root = tk.Tk()
root.title("Password Manager with RSA")

# Username entry
username_label = tk.Label(root, text="Username:")
username_label.grid(row=0, column=0, padx=10, pady=5, sticky=tk.W)
username_entry = tk.Entry(root)
username_entry.grid(row=0, column=1, padx=10, pady=5)

# Unique Key entry
key_label = tk.Label(root, text="Unique Key:")
key_label.grid(row=1, column=0, padx=10, pady=5, sticky=tk.W)
key_entry = tk.Entry(root, show="*")
key_entry.grid(row=1, column=1, padx=10, pady=5)

# Password entry
password_label = tk.Label(root, text="Password:")
password_label.grid(row=2, column=0, padx=10, pady=5, sticky=tk.W)
password_entry = tk.Entry(root, show="*")
password_entry.grid(row=2, column=1, padx=10, pady=5)

# Save and Retrieve buttons
save_button = tk.Button(root, text="Save Password", command=save_password_handler)
save_button.grid(row=3, column=0, padx=10, pady=5)
retrieve_button = tk.Button(root, text="Retrieve Password", command=get_password_handler)
retrieve_button.grid(row=3, column=1, padx=10, pady=5)

# Button to show all saved passwords
show_passwords_button = tk.Button(root, text="Show All Saved Passwords", command=show_all_saved_passwords)
show_passwords_button.grid(row=4, column=0, columnspan=2, padx=10, pady=5)

# Treeview widget to display stored passwords
tree = ttk.Treeview(root, columns=("Username", "Encrypted Password"))
tree.heading("Username", text="Username")
tree.heading("Encrypted Password", text="Encrypted Password")
tree.grid(row=100, column=0, columnspan=2, padx=5, pady=5, sticky=tk.W+tk.E)

root.mainloop()

